# Class-weighted Learning
- Class-imbalance problem is actually a quite common problem. For instance, there are much more purchasers among mobile app users and much more non-criminals than criminals in society.
- However, if class imbalance is too severe (i.e., training set is highly skewed), it is likely to  bear undesirable effects. 
    - For instance, algorithm will tend to vote for majority class, all the time.
    - This is highly risky since we might lose track of purchasers among mobile app users and criminals, which are relatively rare among training instances

In [1]:
import numpy as np
from sklearn.utils import class_weight
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam

Using TensorFlow backend.


## Load Dataset
- Breast cancer dataset in ```sklearn```
- doc: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [2]:
data = load_breast_cancer()
X_data = data.data.tolist()
y_data = data.target.tolist()

In [3]:
print("Number of malignant instances (0): ", Counter(y_data)[0])
print("Number of benign instances (1): ", Counter(y_data)[1])

Number of malignant instances (0):  212
Number of benign instances (1):  357


In [4]:
# delete some of malignant instances to generate class-imbalance situation artificially
for i in range(200):
    if y_data[i] == 0:
        X_data[i] = None
        y_data[i] = None

In [5]:
X_data = [x for x in X_data if x != None]
y_data = [y for y in y_data if y != None]

In [6]:
print("Number of malignant instances (0): ", Counter(y_data)[0])
print("Number of benign instances (1): ", Counter(y_data)[1])

Number of malignant instances (0):  108
Number of benign instances (1):  357


In [7]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X_data), np.asarray(y_data), test_size = 0.2, random_state = 7) 

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(372, 30) (93, 30) (372,) (93,)


## Computing class weights
- We compute class weights based on training dataset, and deliver as parameter when fitting

In [11]:
weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [12]:
class_weights = dict(zip(np.unique(y_train), weights))
print("Computed class weights: ", class_weights)

Computed class weights:  {0: 2.2409638554216866, 1: 0.643598615916955}


## Naive Learning

In [13]:
def simple_mlp():
    model = Sequential()
    model.add(Dense(10, input_shape = (X_train.shape[1],), activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = adam(lr = 0.001), loss = 'binary_crossentropy', metrics = ['acc'])
    return model

In [14]:
model = simple_mlp()
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
372/372 [==============================] - 0s 761us/step - loss: 12.5219 - acc: 0.2231
Epoch 2/100
372/372 [==============================] - 0s 41us/step - loss: 12.5219 - acc: 0.2231
Epoch 3/100
372/372 [==============================] - 0s 42us/step - loss: 12.5219 - acc: 0.2231
Epoch 4/100
372/372 [==============================] - 0s 46us/step - loss: 12.5219 - acc: 0.2231
Epoch 5/100
372/372 [==============================] - 0s 46us/step - loss: 12.5219 - acc: 0.2231
Epoch 6/100
372/372 [==============================] - 0s 47us/step - loss: 12.5219 - acc: 0.2231
Epoch 7/100
372/372 [==============================] - 0s 45us/step - loss: 12.5219 - acc: 0.2231
Epoch 8/100
372/372 [==============================] - 0s 47us/step - loss: 12.5219 - acc: 0.2231
Epoch 9/100
372/372 [==============================] - 0s 45us/step - loss: 12.5219 - acc: 0.2231
Epoch 10/100
372/372 [==============================] - 0s 47us/step - loss: 12.5219 - acc: 0.2231
Epoch 11/100
372/3

372/372 [==============================] - 0s 43us/step - loss: 12.5219 - acc: 0.2231
Epoch 84/100
372/372 [==============================] - 0s 43us/step - loss: 12.5219 - acc: 0.2231
Epoch 85/100
372/372 [==============================] - 0s 39us/step - loss: 12.5219 - acc: 0.2231
Epoch 86/100
372/372 [==============================] - 0s 44us/step - loss: 12.5219 - acc: 0.2231
Epoch 87/100
372/372 [==============================] - 0s 41us/step - loss: 12.5219 - acc: 0.2231
Epoch 88/100
372/372 [==============================] - 0s 41us/step - loss: 12.5219 - acc: 0.2231
Epoch 89/100
372/372 [==============================] - 0s 42us/step - loss: 12.5219 - acc: 0.2231
Epoch 90/100
372/372 [==============================] - 0s 46us/step - loss: 12.5219 - acc: 0.2231
Epoch 91/100
372/372 [==============================] - 0s 45us/step - loss: 12.5219 - acc: 0.2231
Epoch 92/100
372/372 [==============================] - 0s 50us/step - loss: 12.5219 - acc: 0.2231
Epoch 93/100
372/372 [=

In [15]:
y_pred = model.predict(X_test).round()

In [16]:
print("% of predicted 1's: ", y_pred.sum()/len(y_pred))
print("Overall Accuracy Score: ", accuracy_score(y_pred, y_test))

% of predicted 1's:  0.0
Overall Accuracy Score:  0.26881720430107525


## Class-weighted learning

In [17]:
model = simple_mlp()
model.fit(X_train, y_train, epochs = 100, class_weight = class_weights)

Epoch 1/100
372/372 [==============================] - 0s 718us/step - loss: 6.7169 - acc: 0.3737
Epoch 2/100
372/372 [==============================] - 0s 44us/step - loss: 5.7320 - acc: 0.3306
Epoch 3/100
372/372 [==============================] - 0s 47us/step - loss: 4.5548 - acc: 0.5000
Epoch 4/100
372/372 [==============================] - 0s 47us/step - loss: 4.2774 - acc: 0.3548
Epoch 5/100
372/372 [==============================] - 0s 47us/step - loss: 3.3119 - acc: 0.5000
Epoch 6/100
372/372 [==============================] - 0s 52us/step - loss: 2.8563 - acc: 0.6667
Epoch 7/100
372/372 [==============================] - 0s 47us/step - loss: 2.4438 - acc: 0.6398
Epoch 8/100
372/372 [==============================] - 0s 48us/step - loss: 2.2266 - acc: 0.6640
Epoch 9/100
372/372 [==============================] - 0s 49us/step - loss: 2.0798 - acc: 0.7312
Epoch 10/100
372/372 [==============================] - 0s 51us/step - loss: 1.9788 - acc: 0.6774
Epoch 11/100
372/372 [======

372/372 [==============================] - 0s 48us/step - loss: 0.3624 - acc: 0.9140
Epoch 85/100
372/372 [==============================] - 0s 43us/step - loss: 0.2918 - acc: 0.9140
Epoch 86/100
372/372 [==============================] - 0s 43us/step - loss: 0.3377 - acc: 0.8898
Epoch 87/100
372/372 [==============================] - 0s 43us/step - loss: 0.2231 - acc: 0.9220
Epoch 88/100
372/372 [==============================] - 0s 45us/step - loss: 0.2616 - acc: 0.9194
Epoch 89/100
372/372 [==============================] - 0s 45us/step - loss: 0.2783 - acc: 0.9167
Epoch 90/100
372/372 [==============================] - 0s 43us/step - loss: 0.2664 - acc: 0.9247
Epoch 91/100
372/372 [==============================] - 0s 48us/step - loss: 0.2773 - acc: 0.9167
Epoch 92/100
372/372 [==============================] - 0s 42us/step - loss: 0.3613 - acc: 0.8763
Epoch 93/100
372/372 [==============================] - 0s 41us/step - loss: 0.2139 - acc: 0.9247
Epoch 94/100
372/372 [===========

In [18]:
y_pred = model.predict(X_test).round()

In [19]:
print("% of predicted 1's: ", y_pred.sum()/len(y_pred))
print("Overall Accuracy Score: ", accuracy_score(y_pred, y_test))

% of predicted 1's:  0.7096774193548387
Overall Accuracy Score:  0.9354838709677419
